# Metadata for the National Flood Insurance Program (NFIP) Claims Data
Author: Mark Bauer

In [1]:
# import libraries
from datetime import datetime
import json
import requests
import pandas as pd
import duckdb

In [2]:
# reproducibility
%reload_ext watermark
%watermark -v -p json,requests,pandas,duckdb

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.6.0

json    : 2.0.9
requests: 2.28.1
pandas  : 1.5.1
duckdb  : 1.0.0



In [3]:
# data retrieved, not to be confused with data of data
current_date = datetime.now()
print(f"Metadata was retrieved on {current_date.strftime('%Y-%m-%d')}.")

Metadata was retrieved on 2025-05-11.


# OpenFEMA Metadata Endpoints
This notebook focuses on using two key OpenFEMA API endpoints:

- OpenFemaDataSets – a list of available FEMA datasets

- OpenFemaDataSetFields – details about the fields in each dataset

Finally, we save the data dictionary to a CSV file.

More information about the APIs can be found in the [OpenFEMA API Documentation](https://www.fema.gov/about/openfema/api).

# OpenFemaDataSets

In [4]:
# API without dataset filter, all datasts
url = "https://www.fema.gov/api/open/v1/OpenFemaDataSets"
response = requests.get(url)

if response.ok:
    data = response.json()
    print("Top-level keys:", list(data.keys()))
else:
    print(f"Failed to download data. Status code: {response.status_code}")

Top-level keys: ['metadata', 'OpenFemaDataSets']


In [5]:
# preview value type
type(data.get('OpenFemaDataSets', {}))

list

In [6]:
# preview first item in list
data.get('OpenFemaDataSets', {})[0]

{'identifier': 'openfema-8',
 'name': 'DataSetFields',
 'title': 'OpenFEMA Data Set Fields',
 'description': "Metadata for the OpenFEMA API data set fields. It contains descriptions, data types, and other attributes for each field.\n\nCitation: FEMA's citation requirements for datasets (API usage or file downloads) can be found on the OpenFEMA Terms and Conditions page, Citing Data section: https://www.fema.gov/about/openfema/terms-conditions.\n\nFor answers to Frequently Asked Questions (FAQs) about the OpenFEMA program, API, and publicly available datasets, please visit: https://www.fema.gov/about/openfema/faq.\nIf you have media inquiries about this dataset, please email the FEMA Press Office at FEMA-Press-Office@fema.dhs.gov or call (202) 646-3272. For inquiries about FEMA's data and Open Government program, please email the OpenFEMA team at OpenFEMA@fema.dhs.gov.",
 'webService': 'https://www.fema.gov/api/open/v1/OpenFemaDataSetFields',
 'dataDictionary': 'https://www.fema.gov/ope

In [7]:
# number of datasets
len(data.get('OpenFemaDataSets', {}))

41

In [8]:
# create dataframe
datasets_df = pd.DataFrame(data.get('OpenFemaDataSets', {}))

datasets_df.head()

,identifier,name,title,description,webService,dataDictionary,keyword,modified,publisher,contactPoint,...,recordCount,depDate,depApiMessage,depWebMessage,depNewURL,hash,lastRefresh,id,lastDataSetRefresh,distribution
0,openfema-8,DataSetFields,OpenFEMA Data Set Fields,Metadata for the OpenFEMA API data set fields....,https://www.fema.gov/api/open/v1/OpenFemaDataS...,https://www.fema.gov/openfema-data-page/openfe...,"[open, data, fema, datasets, metadata, api]",2019-06-04T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,...,1127.0,None,None,None,None,30ed5254158b5837c3d39b3962a7c086a402f0ad,2024-11-21T14:40:15.168Z,2403d533-6186-4055-ad74-d58cc7138361,2025-04-01T20:00:49.774Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
1,openfema-88,DataSetCodes,OpenFEMA Dataset Codes,Metadata for the OpenFEMA API dataset codes. T...,https://www.fema.gov/api/open/v1/DataSetCodes,https://www.fema.gov/openfema-data-page/openfe...,"[metadata, codes]",2025-02-20T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,...,715.0,None,None,None,None,bdb6e95190d04b9c81c19774f9ec0d51251a5639,2025-03-05T19:40:34.218Z,b45ed075-8c5f-487d-accc-eae7e80cfeb2,2025-04-01T20:01:01.331Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
2,openfema-87,HazardMitigationAssistanceProjectsFinancialTra...,Hazard Mitigation Assistance Projects Financia...,This dataset contains information on the HMA s...,https://www.fema.gov/api/open/v1/HazardMitigat...,https://www.fema.gov/openfema-data-page/hazard...,"[mitigation, hazard, disaster, non-disaster, g...",2024-11-06T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,...,74742.0,None,None,None,None,4ec3558406643d7c9df1d7c71e6080a2cb9fc395,2024-11-21T14:40:15.168Z,e7ce986a-0495-45da-b105-9589ef45a356,2025-05-11T15:07:52.240Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
3,openfema-54,HousingAssistanceOwners,Housing Assistance Program Data - Owners,This dataset is generated by FEMA's Individual...,https://www.fema.gov/api/open/v2/HousingAssist...,https://www.fema.gov/openfema-data-page/housin...,"[housing assistance, owners, disaster]",2024-01-25T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,...,156370.0,None,None,None,None,3ec2afee594f46f86c0f38817ce32c4a8ee862bd,2025-05-10T07:38:12.660Z,9a4b4dbc-87bc-411b-b4fb-a5f6608abaae,2025-05-10T07:38:12.660Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
4,openfema-26,FemaWebDeclarationAreas,FEMA Web Declaration Areas,This data set contains general information on ...,https://www.fema.gov/api/open/v1/FemaWebDeclar...,https://www.fema.gov/openfema-data-page/fema-w...,"[disaster, declaration, fema website]",2023-01-10T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,...,473818.0,None,None,None,None,dc72e3569e7cbb8fdbeaaf404672cab4e322328d,2024-11-21T14:40:15.168Z,3f2975a6-3c3b-4c29-9ad0-3954efab268d,2025-05-12T02:22:13.198Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."


In [9]:
# column info
datasets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 39 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   identifier              41 non-null     object 
 1   name                    41 non-null     object 
 2   title                   41 non-null     object 
 3   description             41 non-null     object 
 4   webService              41 non-null     object 
 5   dataDictionary          41 non-null     object 
 6   keyword                 41 non-null     object 
 7   modified                41 non-null     object 
 8   publisher               41 non-null     object 
 9   contactPoint            41 non-null     object 
 10  mbox                    41 non-null     object 
 11  accessLevel             41 non-null     object 
 12  landingPage             40 non-null     object 
 13  temporal                32 non-null     object 
 14  api                     41 non-null     bool

In [10]:
# slice through columns to preview all of them
datasets_df.iloc[:5, :20]

,identifier,name,title,description,webService,dataDictionary,keyword,modified,publisher,contactPoint,mbox,accessLevel,landingPage,temporal,api,version,bureauCode,programCode,accessLevelComment,license
0,openfema-8,DataSetFields,OpenFEMA Data Set Fields,Metadata for the OpenFEMA API data set fields....,https://www.fema.gov/api/open/v1/OpenFemaDataS...,https://www.fema.gov/openfema-data-page/openfe...,"[open, data, fema, datasets, metadata, api]",2019-06-04T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,openfema@fema.gov,public,https://www.fema.gov/about/reports-and-data/op...,None,True,1,[024:70],[024:039],None,None
1,openfema-88,DataSetCodes,OpenFEMA Dataset Codes,Metadata for the OpenFEMA API dataset codes. T...,https://www.fema.gov/api/open/v1/DataSetCodes,https://www.fema.gov/openfema-data-page/openfe...,"[metadata, codes]",2025-02-20T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,openfema@fema.gov,public,https://www.fema.gov/about/reports-and-data/op...,None,True,1,[024:70],[024:039],None,None
2,openfema-87,HazardMitigationAssistanceProjectsFinancialTra...,Hazard Mitigation Assistance Projects Financia...,This dataset contains information on the HMA s...,https://www.fema.gov/api/open/v1/HazardMitigat...,https://www.fema.gov/openfema-data-page/hazard...,"[mitigation, hazard, disaster, non-disaster, g...",2024-11-06T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,openfema@fema.gov,public,https://www.fema.gov/grants/mitigation,1990-03-17/,True,1,[024:70],[024:039],None,None
3,openfema-54,HousingAssistanceOwners,Housing Assistance Program Data - Owners,This dataset is generated by FEMA's Individual...,https://www.fema.gov/api/open/v2/HousingAssist...,https://www.fema.gov/openfema-data-page/housin...,"[housing assistance, owners, disaster]",2024-01-25T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,openfema@fema.gov,public,https://www.fema.gov/individual-disaster-assis...,2004-08-01/,True,2,[024:70],[024:039],None,None
4,openfema-26,FemaWebDeclarationAreas,FEMA Web Declaration Areas,This data set contains general information on ...,https://www.fema.gov/api/open/v1/FemaWebDeclar...,https://www.fema.gov/openfema-data-page/fema-w...,"[disaster, declaration, fema website]",2023-01-10T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,openfema@fema.gov,public,https://www.fema.gov/disasters,1960-11-01/,True,1,[024:70],[024:039],None,None


In [11]:
datasets_df.iloc[:5, 20:]

,spatial,theme,dataQuality,accrualPeriodicity,language,primaryITInvestmentUII,references,issued,systemOfRecords,recordCount,depDate,depApiMessage,depWebMessage,depNewURL,hash,lastRefresh,id,lastDataSetRefresh,distribution
0,None,Miscellaneous,true,irregular,en-US,None,[],2013-01-01T00:00:00.000Z,None,1127.0,None,None,None,None,30ed5254158b5837c3d39b3962a7c086a402f0ad,2024-11-21T14:40:15.168Z,2403d533-6186-4055-ad74-d58cc7138361,2025-04-01T20:00:49.774Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
1,None,Miscellaneous,true,irregular,en-US,None,[],2025-03-05T00:00:00.000Z,None,715.0,None,None,None,None,bdb6e95190d04b9c81c19774f9ec0d51251a5639,2025-03-05T19:40:34.218Z,b45ed075-8c5f-487d-accc-eae7e80cfeb2,2025-04-01T20:01:01.331Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
2,None,Hazard Mitigation,true,R/P1D,en-US,None,[],2024-11-08T00:00:00.000Z,None,74742.0,None,None,None,None,4ec3558406643d7c9df1d7c71e6080a2cb9fc395,2024-11-21T14:40:15.168Z,e7ce986a-0495-45da-b105-9589ef45a356,2025-05-11T15:07:52.240Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
3,None,Individual Assistance,true,R/P1W,en-US,None,[],2013-06-26T00:00:00.000Z,None,156370.0,None,None,None,None,3ec2afee594f46f86c0f38817ce32c4a8ee862bd,2025-05-10T07:38:12.660Z,9a4b4dbc-87bc-411b-b4fb-a5f6608abaae,2025-05-10T07:38:12.660Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
4,None,Disaster Information,true,R/PT20M,en-US,None,[],2013-05-01T00:00:00.000Z,None,473818.0,None,None,None,None,dc72e3569e7cbb8fdbeaaf404672cab4e322328d,2024-11-21T14:40:15.168Z,3f2975a6-3c3b-4c29-9ad0-3954efab268d,2025-05-12T02:22:13.198Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."


In [12]:
# number of datasets per theme
datasets_df['theme'].value_counts()

Hazard Mitigation                           10
National Flood Insurance Program             8
Disaster Information                         6
Individual Assistance                        5
Public Assistance                            5
Miscellaneous                                4
Emergency Management/Preparedness/Alerts     3
Name: theme, dtype: int64

In [13]:
# preview NFIP datasets
datasets_df.loc[datasets_df['theme'] == 'National Flood Insurance Program']

,identifier,name,title,description,webService,dataDictionary,keyword,modified,publisher,contactPoint,...,recordCount,depDate,depApiMessage,depWebMessage,depNewURL,hash,lastRefresh,id,lastDataSetRefresh,distribution
14,openfema-75,NfipCommunityLayerNoOverlapsWhole,NFIP Community Layer No Overlaps Whole,This dataset is flattened and multicounty comm...,https://www.fema.gov/api/open/v1/NfipCommunity...,https://www.fema.gov/openfema-data-page/nfip-c...,"[nfip community, geospatial, census, boundaries]",2024-02-22T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,...,29898.0,None,None,None,None,7bdabebd0a0805adbb76fee3ca61bbd4a5f5c0b7,2024-12-05T22:20:23.322Z,e2c2268c-7a3c-4ce1-830e-ac5f735ee44c,2024-12-05T18:10:50.474Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
16,openfema-77,NfipCommunityLayerComprehensive,NFIP Community Layer Comprehensive,This dataset is also known as the 3d layer and...,https://www.fema.gov/api/open/v1/NfipCommunity...,https://www.fema.gov/openfema-data-page/nfip-c...,"[nfip community, geospatial, census, boundaries]",2024-02-22T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,...,51943.0,None,None,None,None,51551de69709270cbb3dc05a9da88e92be099c2d,2024-12-05T22:20:23.322Z,0131d365-5a18-47a0-98cb-558149c165ee,2024-12-05T18:09:51.898Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
17,openfema-76,NfipCommunityLayerNoOverlapsSplit,NFIP Community Layer No Overlaps Split,This dataset is flattened and multicounty comm...,https://www.fema.gov/api/open/v1/NfipCommunity...,https://www.fema.gov/openfema-data-page/nfip-c...,"[nfip community, geospatial, census, boundaries]",2024-02-22T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,...,37918.0,None,None,None,None,8db6fcd098f2b2bf16a3dc6b10ccb93543d32010,2024-12-05T22:20:23.322Z,57d33198-ea17-4255-83fe-35742b8f601b,2024-12-05T18:09:57.312Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
21,openfema-74,FimaNfipPolicies,FIMA NFIP Redacted Policies,Congress passed the National Flood Insurance A...,https://www.fema.gov/api/open/v2/FimaNfipPolicies,https://www.fema.gov/openfema-data-page/fima-n...,"[NFIP, Flood, Insurance, Policy, Water]",2025-04-01T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,...,69489458.0,None,None,None,None,85c6b91e237853b3333b06b4e3ebf5cdef930060,2025-04-01T20:00:37.246Z,3776bead-efe2-4590-84eb-3e000f869701,2025-04-05T00:14:18.878Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
22,openfema-86,NfipResidentialPenetrationRates,NFIP Residential Penetration Rates,"NFIP ""take-up rates"" or residential penetratio...",https://www.fema.gov/api/open/v1/NfipResidenti...,https://www.fema.gov/openfema-data-page/nfip-r...,"[nfip, rates, nsi, residential structures, pol...",2024-11-07T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,...,3145.0,None,None,None,None,c84151b7bbe46b75c5143a0c9d491a9986247f06,2024-11-21T14:40:15.168Z,18f6002e-40a5-4601-9504-279b8e6bf985,2025-01-02T23:01:15.547Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
24,openfema-73,FimaNfipClaims,FIMA NFIP Redacted Claims,Congress passed the National Flood Insurance A...,https://www.fema.gov/api/open/v2/FimaNfipClaims,https://www.fema.gov/openfema-data-page/fima-n...,"[NFIP, FIMA, claims, flood insurance]",2023-03-24T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,...,2709121.0,None,None,None,None,843979427d2c4cc20e139ce661e6b0476975431f,2024-11-21T14:40:15.168Z,e9343760-d76e-44db-b947-822642353e7f,2025-04-04T23:23:26.173Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
25,openfema-78,NfipMultipleLossProperties,NFIP Multiple Loss Properties,This dataset provides information on structure...,https://www.fema.gov/api/open/v1/NfipMultipleL...,https://www.fema.gov/openfema-data-page/NFIP-M...,"[NFIP, Insurance, Flood, Repetitive Loss, Dama...",2024-02-27T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,...,257374.0,None,None,None,None,c6474d2bed656468b54afcb63f3da841959535a5,2024-11-21T14:40:15.168Z,f

We use a filter option to only return metadata about the NFIP Claims dataset.

In [14]:
url = "https://www.fema.gov/api/open/v1/OpenFemaDataSets?$filter=name%20eq%20%27FimaNfipClaims%27"
response = requests.get(url)

if response.ok:
    data = response.json()
    print("Top-level keys:", list(data.keys()))
else:
    print(f"Failed to download data. Status code: {response.status_code}")

Top-level keys: ['metadata', 'OpenFemaDataSets']


In [15]:
print(json.dumps(data.get('metadata', {}), indent=2))

{
  "skip": 0,
  "filter": "name eq 'FimaNfipClaims'",
  "orderby": "",
  "select": null,
  "rundate": "2025-05-12T02:28:54.906Z",
  "top": 1000,
  "format": "json",
  "metadata": true,
  "entityname": "OpenFemaDataSets",
  "version": "v1",
  "url": "/api/open/v1/OpenFemaDataSets?$filter=name%20eq%20%27FimaNfipClaims%27",
  "count": 0
}


In [16]:
print(json.dumps(data.get('OpenFemaDataSets', {}), indent=2))

[
  {
    "identifier": "openfema-73",
    "name": "FimaNfipClaims",
    "title": "FIMA NFIP Redacted Claims",
    "description": "Congress passed the National Flood Insurance Act (NFIA), 42 U.S.C. 4001 in 1968, creating the National Flood Insurance Program (NFIP) in order to reduce future flood losses through flood hazard identification, floodplain management, and providing insurance protection. The Department of Housing and Urban Development (HUD) originally administered the NFIP, and Congress subsequently transferred the NFIP to FEMA upon its creation in 1979. FEMA and insurance companies participating in FEMA's Write Your Own (WYO) program offer NFIP insurance coverage for building structures as well as for contents and personal property within the building structures to eligible and insurable properties. The WYO program began in 1983 with NFIP operating under Part B of the NFIA and allows FEMA to authorize private insurance companies to issue the Standard Flood Insurance Policy (S

# OpenFemaDataSetFields
We use the filter option to directly return the NFIP Claims dataset information.

In [17]:
url = "https://www.fema.gov/api/open/v1/OpenFemaDataSetFields?$filter=openFemaDataSet%20eq%20%27FimaNfipClaims%27%20and%20datasetVersion%20eq%202"
response = requests.get(url)

if response.ok:
    data = response.json()
    print("Top-level keys:", list(data.keys()))
else:
    print(f"Failed to download data. Status code: {response.status_code}")

Top-level keys: ['metadata', 'OpenFemaDataSetFields']


In [18]:
# preview metadata key
print(json.dumps(data.get('metadata', {}), indent=2))

{
  "skip": 0,
  "filter": "openFemaDataSet eq 'FimaNfipClaims' and datasetVersion eq 2",
  "orderby": "",
  "select": null,
  "rundate": "2025-05-12T02:28:55.327Z",
  "top": 1000,
  "format": "json",
  "metadata": true,
  "entityname": "OpenFemaDataSetFields",
  "version": "v1",
  "url": "/api/open/v1/OpenFemaDataSetFields?$filter=openFemaDataSet%20eq%20%27FimaNfipClaims%27%20and%20datasetVersion%20eq%202",
  "count": 0
}


In [19]:
# examine dataset fields key
print(json.dumps(data.get('OpenFemaDataSetFields', {}), indent=2))

[
  {
    "datasetId": "openfema-73",
    "openFemaDataSet": "FimaNfipClaims",
    "datasetVersion": 2,
    "name": "asOfDate",
    "title": "As of Date",
    "description": "The effective date of the data in the file. In other words, the date when the record was created or last updated in the source system. When time is not specified in the field, it will default to T00:00:00.000Z",
    "type": "datetime",
    "sortOrder": 2,
    "isSearchable": true,
    "isNestedObject": false,
    "isNullable": true,
    "primaryKey": false,
    "id": "97719ad8-6820-46ef-baca-5ac81be4ce30",
    "lastRefresh": "2024-11-07T21:42:54.493Z",
    "hash": "5ebd8717745887e4f21f71f3107339b807b7a75b",
    "srid": null
  },
  {
    "datasetId": "openfema-73",
    "openFemaDataSet": "FimaNfipClaims",
    "datasetVersion": 2,
    "name": "agricultureStructureIndicator",
    "title": "Agriculture Structure Indicator",
    "description": "Indicates whether a building is reported as being an agricultural structure

In [20]:
# create a dataframe
df = pd.DataFrame(data.get('OpenFemaDataSetFields', {}))

# preview number columns, rows
print(df.shape)
df

(73, 16)


,datasetId,openFemaDataSet,datasetVersion,name,title,description,type,sortOrder,isSearchable,isNestedObject,isNullable,primaryKey,id,lastRefresh,hash,srid
0,openfema-73,FimaNfipClaims,2,asOfDate,As of Date,The effective date of the data in the file. In...,datetime,2,True,False,True,False,97719ad8-6820-46ef-baca-5ac81be4ce30,2024-11-07T21:42:54.493Z,5ebd8717745887e4f21f71f3107339b807b7a75b,None
1,openfema-73,FimaNfipClaims,2,agricultureStructureIndicator,Agriculture Structure Indicator,Indicates whether a building is reported as be...,boolean,1,False,False,True,False,2fec7d27-8783-412d-a518-cbb24327f2e5,2024-08-27T19:21:04.454Z,1a7fb8b860c960b627097b2f8a5ff5f1fcbaf7bf,None
2,openfema-73,FimaNfipClaims,2,basementEnclosureCrawlspaceType,Basement Enclosure Crawlspace Type,Basement is defined for purposes of the NFIP a...,smallint,3,True,False,True,False,d17676e1-00be-4fde-9a26-1dee3c41f09b,2024-08-27T19:21:04.454Z,d2aed3f3ab86e2340999b6c9a5482c779afa74be,None
3,openfema-73,FimaNfipClaims,2,policyCount,Policy Count,Insured units in an active status. A policy co...,smallint,4,True,False,True,False,647a6e15-04cb-400a-be9b-cafcfd0168a9,2024-08-27T19:21:04.454Z,036a5b3b87320834f7b2635e81bcfe8f4c03fd44,None
4,openfema-73,FimaNfipClaims,2,crsClassificationCode,CRS Classification Code,The Community Rating System (CRS) Classificati...,smallint,5,True,False,True,False,c7adf0e8-c8cb-4a63-bb4a-11387f864194,2024-08-27T19:21:04.454Z,383c80bbc9593a83add82547fa51210e0d4eb6cc,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,openfema-73,FimaNfipClaims,2,censusTract,Census Tract,U.S. Census Bureau defined census tracts; stat...,text,69,True,False,True,False,b7948f15-f64e-406e-8028-4f6da8c8f48f,2024-08-27T19:21:04.454Z,0f5a536267c5f8a00dacd63ab1d8c71731728d64,None
69,openfema-73,FimaNfipClaims,2,censusBlockGroupFips,Census Block Group FIPS,The first digit of the census block number ide...,text,70,True,False,True,False,b5b80826-065e-44aa-98ee-65a8a3aa3c99,2024-08-27T19:21:04.454Z,79c1dde3b0b137dfa8eebed442b97951e440bbb2,None
70,openfema-73,FimaNfipClaims,2,latitude,Latitude,Approximate latitude of the insured building (...,"decimal(9,1)",71,True,False,True,False,a3ffbbc7-2af4-47e8-8406-19aa0038fefa,2024-08-27T19:21:04.454Z,0a546b9587f2617a3bdee20255bfa3ec20bcbb1c,None
71,openfema-73,FimaNfipClaims,2,longitude,Longitude,Approximate longitude of the insured building ...,"decimal(9,1)",72,True,False,True,False,5f36c9c0-7ca2-44ab-843a-c251f872bdb2,2024-08-27T19:21:04.454Z,245fe176844bc4300e45ca404a1ec9909f694eb9,None


# Write Out Data Dictionary as a CSV

In [21]:
df.to_csv("data-dictionary.csv", index=False)

%ls

LICENSE              data/                figures/
README.md            data-dictionary.csv  figures.ipynb
analysis.ipynb       download-data.ipynb  metadata.ipynb
